In [1]:
import tensorflow as tf
import pandas as pd
import datetime
from pandas_profiling import ProfileReport

In [3]:
# Read movement data, this could be from several locations like S3 buckets
movements = pd.read_csv("../data/SmartMovementExport.csv")

In [4]:
movements.head()

,id,acceloX,acceloY,acceloZ,userAcceloX,userAcceloY,userAcceloZ,gyroX,gyroY,gyroZ,lightSensor,locked
0,2020-11-14 22:18:31.510956,-2.109356,2.882707,9.466963,0.218186,0.537032,0.203292,-0.711324,-0.419993,0.339640,8,0
1,2020-11-14 22:18:31.558048,-2.109356,2.882707,9.466963,0.676335,0.776491,0.463521,-0.711324,-0.419993,0.339640,8,0
2,2020-11-14 22:18:31.558393,-2.109356,2.882707,9.466963,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,8,0
3,2020-11-14 22:18:31.558616,-3.510007,1.884294,9.316123,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,8,0
4,2020-11-14 22:18:31.669076,-3.510007,1.884294,9.316123,0.676335,0.776491,0.463521,-1.693300,-1.869679,0.440446,7,0


,Timepoint,Longitude,Latitude,AcceloX,AcceloY,AcceloZ,GyroX,GyroY,GyroZ,Lux,Unlocked
0,1594410747,52.516235,13.377372,0,0.5,0.5,-1.0,0,0,0,0
1,1594411000,52.516255,13.376530,0,0.6,0.6,-1.0,0,0,0,0
2,1594411100,52.516094,13.373927,0,0.7,0.7,-1.0,0,0,0,0
3,1594412000,52.515999,13.372479,0,0.8,0.8,-0.8,0,0,0,0
4,1594421000,52.515891,13.370706,0,0.9,0.9,-0.6,0,0,6,0


In [5]:
CSV_COLUMN_NAMES = ['id', 'acceloX', 'acceloY', 'acceloZ', 'userAcceloX', 'userAcceloY','userAcceloZ', 'gyroX', 'gyroY', 'gyroZ', 'lightSensot', 'locked']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(movements, movements['locked'], test_size=0.33, random_state=42)

Explore the smart movement data with PandasProfiling


In [7]:
profile = ProfileReport(movements, title="Pandas Profiling Report")



In [8]:
profile.to_widgets()


Generate report structure: 100%|██████████| 1/1 [00:03<00:00,  3.08s/it]
